In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset = pd.get_dummies(dataset,dtype=int, drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [10]:
independent=dataset[['Age', 'EstimatedSalary', 'Gender_Male']]

In [11]:
independent.shape

(400, 3)

In [12]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [13]:
dependent=dataset[[ 'Purchased',]]

In [14]:
from sklearn.model_selection import train_test_split    
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [15]:
from sklearn.naive_bayes import CategoricalNB

In [17]:
from sklearn.model_selection import GridSearchCV               
param_grid = {'alpha':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
             'force_alpha':[True,False],
             'fit_prior':[True,False],
             'class_prior':[None],
             'min_categories':[None]}
grid = GridSearchCV(CategoricalNB(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
grid.fit(x_train,y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\sabeen\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan]
  warnings.warn(
C:\sabeen\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=CategoricalNB(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,
                                   1.0],
                         'class_prior': [None], 'fit_prior': [True, False],
                         'force_alpha': [True, False],
                         'min_categories': [None]},
             scoring='f1_weighted', verbose=3)

In [18]:
re=grid.cv_results_

grid_prediction=grid.predict(x_test)



In [19]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_prediction)
print(cm)

[[69 10]
 [11 30]]


In [20]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_prediction)
print(clf_report)

              precision    recall  f1-score   support

           0       0.86      0.87      0.87        79
           1       0.75      0.73      0.74        41

    accuracy                           0.82       120
   macro avg       0.81      0.80      0.80       120
weighted avg       0.82      0.82      0.82       120



In [21]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_prediction,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'alpha': 0.1, 'class_prior': None, 'fit_prior': True, 'force_alpha': True, 'min_categories': None}: 0.8244700675518286


In [22]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[69 10]
 [11 30]]


In [23]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.86      0.87      0.87        79
           1       0.75      0.73      0.74        41

    accuracy                           0.82       120
   macro avg       0.81      0.80      0.80       120
weighted avg       0.82      0.82      0.82       120



In [24]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])


0.883297313985798

In [25]:
table=pd.DataFrame.from_dict(re)
table


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_class_prior,param_fit_prior,param_force_alpha,param_min_categories,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.013220,0.006536,0.013632,0.006001,0.1,None,True,True,None,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.753521,0.690193,0.730543,0.792857,NaN,NaN,NaN,1
1,0.019423,0.001667,0.008514,0.004434,0.1,None,True,False,None,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.753521,0.690193,0.730543,0.792857,NaN,NaN,NaN,1
2,0.017962,0.002668,0.008027,0.003010,0.1,None,False,True,None,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.775048,0.690193,0.752381,0.855556,NaN,NaN,NaN,1
3,0.015799,0.001885,0.009631,0.001886,0.1,None,False,False,None,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.775048,0.690193,0.752381,0.855556,NaN,NaN,NaN,1
4,0.017280,0.001743,0.007858,0.001065,0.2,None,True,True,None,"{'alpha': 0.2, 'class_prior': None, 'fit_prior...",0.731250,0.684601,0.730543,0.792857,NaN,NaN,NaN,1
5,0.017104,0.001798,0.008510,0.002675,0.2,None,True,False,None,"{'alpha': 0.2, 'class_prior': None, 'fit_prior...",0.731250,0.684601,0.730543,0.792857,NaN,NaN,NaN,1
6,0.015704,0.002371,0.008821,0.003344,0.2,None,False,True,None,"{'alpha': 0.2, 'class_prior': None, 'fit_prior...",0.775048,0.690193,0.752381,0.855556,NaN,NaN,NaN,1
7,0.019014,0.001548,0.006392,0.003274,0.2,None,False,False,None,"{'alpha': 0.2, 'class_prior': None, 'fit_prior...",0.775048,0.690193,0.752381,0.855556,NaN,NaN,NaN,1
8,0.014045,0.002562,0.008801,0.002874,0.3,None,True,True,None,"{'alpha': 0.3, 'class_prior': None, 'fit_prior...",0.753521,0.684601,0.746799,0.792857,NaN,NaN,NaN,1
9,0.017419,0.002111,0.007342,0.003987,0.3,None,True,False,None,"{'alpha': 0.3, 'class_prior': None, 'fit_prior...",0.753521,0.684601,0.746799,0.792857,NaN,NaN,NaN,1
